In [15]:
import numpy as np
import pandas as pd
import cv2 as cv

Открываем изображение и переводим его в грейскейл

In [2]:
img = cv.imread('flex-1.png')

gsc = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

cv.imwrite('gsc.png', gsc)

True

Проверим, пиксели с какой интенсивностью есть.

In [3]:
np.unique(gsc)

array([168, 255], dtype=uint8)

Бинаризируем изображение

In [4]:
ret,bin = cv.threshold(gsc,240,255,cv.THRESH_BINARY_INV)

cv.imwrite('bin.png', bin)

True

Первично определим все контуры

In [5]:
edges, _ = cv.findContours(bin, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

print(len(edges))

122177


Посмотрим на количества вершин в объектах

In [6]:
array = np.array([len(i) for i in edges])

x, y = np.unique(array, return_counts= True)

print(x, '\n', y)

[  4   6   7   8   9  10  11  12  13  14  15  16  18  19  20  22  23  24
  25  26  27  28  31  40  55  56  59  60  61  65  66  67  69 155 157 158
 266 555] 
 [14181 92622  9632  2934   604   354    28    59    41    26     3  1292
   283     4     2     7     4     8    20     3    26     7     1     8
     2     1     2     2     2     3     2     2     1     1     7     1
     1     1]


Отрисуем и зальем белым только те объекты, у которых количество вершин < 24. Число было выбрано "С запасом". 

In [7]:
black_img = np.zeros((10000, 10000, 1), np.uint8)
counter = 0
for item in edges:
    if len(item) < 24:
        counter += 1
        black_img = cv.drawContours(black_img,  contours=[item], contourIdx=-1, color=(255, 255, 255),  thickness=-1)
print(counter)

cv.imwrite('filled.png', black_img)

122076


True

На получившемся изображении найдем крнутры объектов

In [9]:
def vertices_destroyer(array):
    i_verts_to_del = np.array([], dtype=int)

    for i in range(len(array)-1):
        if abs(array[i][0][0]-array[i+1][0][0]) == 1 and abs(array[i][0][1]-array[i+1][0][1]) == 1:

            i_verts_to_del = np.append(i_verts_to_del, [i])

#Усредняем пиксели на "стыках"
    array[2][0][0] += 1
    array[6][0][1] -= 1
    array[10][0][0] -= 1
    array[14][0][1] += 1
            
    # print(i_verts_to_del)
    array = np.delete(array, i_verts_to_del, axis=0)
    return array

In [11]:
def gaussian_area(_edges):
    sum_1 = sum_2 = 0
    n = len(_edges) - 1

    for i in range(0, n-1):
        sum_1 += _edges[i][0][0] * _edges[i+1][0][1]
        sum_2 -= _edges[i+1][0][0] * _edges[i][0][1]
    
    sum_1 += _edges[n][0][0] * _edges[0][0][1]
    sum_2 -= _edges[0][0][0] * _edges[n][0][1]

    return abs(sum_1 - sum_2) / (2*10000)

In [12]:
def get_side_length(_edges):
    return _edges[1][0][1] - _edges[0][0][1]

In [13]:
new_img = black_img.copy()

background = np.zeros((10000, 10000, 1), np.uint8)

edges2, _ = cv.findContours(new_img, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)


for item in edges2:
    if len(item) == 16:
        item = vertices_destroyer(item)

        # area = gaussian_area(item)
        length = get_side_length(item)

        if  length == 87 or length==88:
            background = cv.drawContours(background,  contours=[item], contourIdx=-1, color=(255, 255, 255),  thickness=-1)

    elif len(item) == 10:
        length = get_side_length(item)

        if length == 269 or length == 270:
            background = cv.drawContours(background,  contours=[item], contourIdx=-1, color=(255, 255, 255),  thickness=-1)
        
cv.imwrite('res.png', background)

True

In [42]:

tmp = background.copy()

edges3, _ = cv.findContours(tmp, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)


In [43]:
left_upper = np.array([np.zeros((1, 2), np.uint8)])
right_bottom = np.array([np.zeros((1, 2), np.uint8)])

In [44]:
for item in edges3:
    if len(item) == 16:
        left_upper = np.append(left_upper, [[item[1][0]]], axis=1)

In [45]:
for item in edges3:
    if len(item) == 16:
        right_bottom = np.append(right_bottom, [[item[9][0]]], axis=1)
    if len(item) == 10:
        right_bottom = np.append(right_bottom, [[item[3][0]]], axis=1)

In [46]:
# print(right_bottom[0][6][0] - left_upper[0][1][0])
# print(right_bottom[0][6][1] - left_upper[0][1][1])

In [47]:
# for i in range(1, len(left_upper[0])):
#     x = left_upper[0][i][0]
#     y = left_upper[0][i][1]
#     img = cv.rectangle(img, (x - 1559, y - 1006), (x, y), (0, 255, 0), 3)

# cv.imwrite('res.png', img)

In [48]:
# for i in range(1, len(left_upper[0])):
#     x1 = right_bottom[0][i+7][0]
#     y1 = right_bottom[0][i+7][1]
#     x2 = left_upper[0][i][0]
#     y2 = left_upper[0][i][1]

#     if abs(x1 - x2) > 1558 or abs(x1 - x2) < 1561 and abs(y1 - y2) > 1005 or abs(y1 - y2) < 1007:
#         img = cv.rectangle(img, right_bottom[0][i+7], left_upper[0][i], (0, 255, 0), 3)

# cv.imwrite('res.png', img)


In [49]:

# for i in range(len(left_upper[0])-6):
#     img = cv.circle(img, left_upper[0][i+1], 10, (0, 255, 0), -1)
#     img = cv.circle(img, right_bottom[0][i+8], 10, (255, 0, 0), 1)

# img = cv.rectangle(img, right_bottom[0][1], right_bottom[0][8], (0, 255, 0), 3)

# cv.imwrite('res.png', img)


In [50]:
print(edges3)

(array([[[9811, 8893]],

       [[9811, 8979]],

       [[9810, 8980]],

       [[9723, 8980]],

       [[9723, 9075]],

       [[9810, 9075]],

       [[9811, 9076]],

       [[9811, 9162]],

       [[9905, 9162]],

       [[9905, 9076]],

       [[9906, 9075]],

       [[9992, 9075]],

       [[9992, 8980]],

       [[9906, 8980]],

       [[9905, 8979]],

       [[9905, 8893]]], dtype=int32), array([[[8157, 8893]],

       [[8157, 8979]],

       [[8156, 8980]],

       [[8070, 8980]],

       [[8070, 9075]],

       [[8156, 9075]],

       [[8157, 9076]],

       [[8157, 9162]],

       [[8252, 9162]],

       [[8252, 9076]],

       [[8253, 9075]],

       [[8339, 9075]],

       [[8339, 8980]],

       [[8253, 8980]],

       [[8252, 8979]],

       [[8252, 8893]]], dtype=int32), array([[[6504, 8893]],

       [[6504, 8979]],

       [[6503, 8980]],

       [[6417, 8980]],

       [[6417, 9075]],

       [[6503, 9075]],

       [[6504, 9076]],

       [[6504, 9162]],

       [[65

In [51]:
print((edges3[0][7][0][1] - edges3[0][0][0][1])/2)
print((edges3[0][15][0][0] - edges3[0][0][0][0])/2)

134.5
47.0


In [52]:
x = (edges3[0][15][0][0] - edges3[0][0][0][0]) / 2 + edges3[0][0][0][0]
y = (edges3[0][7][0][1] - edges3[0][0][0][1]) / 2 + edges3[0][0][0][1]

print(x, y)
img = cv.circle(img, [int(x), int(y)], 10, (0, 255, 0), -1)
cv.imwrite('res.png', img)


9858.0 9027.5


True

In [118]:
centers = np.array([np.zeros((1, 2), np.uint8)])
centers_t_like = np.array([np.zeros((1, 2), np.uint8)])

In [119]:
for item in edges3:
    if len(item) == 10:
        x = int((item[1][0][0] - item[0][0][0]) / 2 + item[0][0][0])
        y = int((item[5][0][1] - item[0][0][1]) / 2 + item[0][0][1])
        centers_t_like = np.append(centers_t_like, [[[x, y]]], axis=1)

In [120]:
for item in edges3:
    if len(item) == 16:
        x = int((item[15][0][0] - item[0][0][0]) / 2 + item[0][0][0])
        y = int((item[7][0][1] - item[0][0][1]) / 2 + item[0][0][1])
        centers = np.append(centers, [[[x, y]]], axis=1)

In [55]:
print(edges3)

(array([[[9811, 8893]],

       [[9811, 8979]],

       [[9810, 8980]],

       [[9723, 8980]],

       [[9723, 9075]],

       [[9810, 9075]],

       [[9811, 9076]],

       [[9811, 9162]],

       [[9905, 9162]],

       [[9905, 9076]],

       [[9906, 9075]],

       [[9992, 9075]],

       [[9992, 8980]],

       [[9906, 8980]],

       [[9905, 8979]],

       [[9905, 8893]]], dtype=int32), array([[[8157, 8893]],

       [[8157, 8979]],

       [[8156, 8980]],

       [[8070, 8980]],

       [[8070, 9075]],

       [[8156, 9075]],

       [[8157, 9076]],

       [[8157, 9162]],

       [[8252, 9162]],

       [[8252, 9076]],

       [[8253, 9075]],

       [[8339, 9075]],

       [[8339, 8980]],

       [[8253, 8980]],

       [[8252, 8979]],

       [[8252, 8893]]], dtype=int32), array([[[6504, 8893]],

       [[6504, 8979]],

       [[6503, 8980]],

       [[6417, 8980]],

       [[6417, 9075]],

       [[6503, 9075]],

       [[6504, 9076]],

       [[6504, 9162]],

       [[65

In [56]:
for i in range(0, len(centers[0])-5):
    x = centers[0][i][0]
    y = centers[0][i][1]
    img = cv.rectangle(img, (x - 1654, y - 1102), (x, y), (0, 255, 0), 3)

cv.imwrite('res.png', img)

True

---

In [129]:
img = cv.imread('cross.png')

gsc = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

edges, _ = cv.findContours(gsc, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

len(edges[0])

654

In [130]:
img = cv.imread('cross_2.png')

gsc = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

edges_t, _ = cv.findContours(gsc, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

len(edges_t[0])

409

In [131]:
# img = cv.imread('filled.png', cv.IMREAD_GRAYSCALE)
img = np.zeros((10000, 10000, 1), np.uint8)

In [132]:
for i in range(0, len(centers[0]-1)):
    arr = np.copy(edges)
    x = centers[0][i][0]
    y = centers[0][i][1]
    for item in arr[0]:
        item[0][0] = item[0][0] - 512 + x
        item[0][1] = item[0][1] - 512 + y
    img = cv.drawContours(img,  contours=arr, contourIdx=-1, color=(255, 255, 255),  thickness=-1)

for i in range(0, len(centers_t_like[0])):
    arr = np.copy(edges_t)
    x = centers_t_like[0][i][0]
    y = centers_t_like[0][i][1]
    for item in arr[0]:
        item[0][0] = item[0][0] - 512 + x
        item[0][1] = item[0][1] - 512 + y
    img = cv.drawContours(img,  contours=arr, contourIdx=-1, color=(255, 255, 255),  thickness=-1)


cv.imwrite('res.png', img)


True

---

In [22]:
img = cv.imread('img.png')

gsc = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

edges, _ = cv.findContours(gsc, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)


In [27]:
def get_square(_edges):
    n = len(_edges)
    dot_1 = _edges[0][0][0]
    dot_3 = _edges[int(n/2)][0][0]

    dot_2 = _edges[int(n/4)][0][1]
    dot_4 = _edges[int(n/4*3)][0][1]

    
    # return [_edges[0][0], _edges[int(n/4*3)][0]]
    return abs((dot_3 - dot_1) * (dot_4 - dot_2))

In [24]:
def get_center(_edges):

    n = len(_edges)
    dot_1 = _edges[0][0][0]
    dot_3 = _edges[int(n/2)][0][0]

    dot_2 = _edges[int(n/4)][0][1]
    dot_4 = _edges[int(n/4*3)][0][1]

    return [int((dot_3-dot_1)/2 + dot_1), int((dot_4-dot_2)/2 + dot_2)]


In [31]:
img = cv.imread('img.png')

array = np.array([])

for i, item in enumerate(edges):
    area = get_square(item)
    print(area)
    array = np.append(array, int(area))
    if area == 187425:
        img = cv.drawContours(img,  contours=[item], contourIdx=-1, color=(0, 0, 255),  thickness=-1)



cv.imwrite('res.png', img)

490
490
490
490
187425
490
490
490
490
187425
490
490
490
490
187425
490
490
490
490
187425
490
490
490
490
187425
490
490
490
490
187425
490
490
490
490
187425
490
490
490
490
187425


True

In [30]:
np.unique(array, return_counts=True)

(array([   490., 187425.]), array([32,  8], dtype=int64))